In [ ]:
!pip install pymysql

In [ ]:
import urllib 
from datetime import date 
from datetime import timedelta 
from datetime import datetime 
import numpy as np 
import pandas as pd 
import pymysql
from sqlalchemy import create_engine

In [ ]:
## READ DATA

# UID_ISO_FIPS_LookUp_Table
csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"
UID_ISO_FIPS_LookUp_Table = pd.read_csv(csv_url)

# time_series_covid19_confirmed_global
csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
time_series_covid19_confirmed_global = pd.read_csv(csv_url)

# time_series_covid19_deaths_global
csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
time_series_covid19_deaths_global = pd.read_csv(csv_url)

# time_series_covid19_recovered_global
csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
time_series_covid19_recovered_global = pd.read_csv(csv_url)

# time_series_covid19_confirmed_US
csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
time_series_covid19_confirmed_US = pd.read_csv(csv_url)

# time_series_covid19_deaths_US
csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
time_series_covid19_deaths_US = pd.read_csv(csv_url)

# daily_reports
today = date.today()
today_str = today.strftime('%m-%d-%Y')
day_delta = timedelta(days=1)
while True:
    try:
        csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv"
        csv_url = csv_url.format(today_str)
        daily_report = pd.read_csv(csv_url)
        print("Daily report for {} is available.".format(today_str))
        break
    except urllib.error.HTTPError:
        print("Daily report for {} is still unavailable.".format(today_str))
        today -= day_delta
        today_str = today.strftime('%m-%d-%Y')
        print("Now we'll try {}.".format(today_str))

Daily report for 07-26-2020 is still unavailable.
Now we'll try 07-25-2020.
Daily report for 07-25-2020 is available.


In [ ]:
# Conditionals
admin2_is_not_null = pd.notnull(daily_report['Admin2'])
admin2_is_null = pd.isnull(daily_report['Admin2'])
province_state_is_not_null = pd.notnull(daily_report['Province_State'])
province_state_is_null = pd.isnull(daily_report['Province_State'])

# Admin2 is not null
daily_report_admin2 = daily_report[admin2_is_not_null]

# Admin2 is null and Province_State is not null
daily_report_province_state = daily_report[admin2_is_null & province_state_is_not_null]

# Admin2 is null and Province_State is null
daily_report_country_region = daily_report[admin2_is_null & province_state_is_null]

In [ ]:
##Data Transposition for global data

#confirmed_global
idVars = ['Province/State', 'Country/Region', 'Lat', 'Long']
time_series_covid19_confirmed_global_long = pd.melt(time_series_covid19_confirmed_global, id_vars=idVars, var_name='Date', value_name='Case')
time_series_covid19_confirmed_global_long.insert(0, 'CaseType', 'confirmed')

#deaths_global
time_series_covid19_deaths_global_long = pd.melt(time_series_covid19_deaths_global, id_vars=idVars, var_name='Date', value_name='Case')
time_series_covid19_deaths_global_long.insert(0, 'CaseType', 'deaths')

#recovered_global
time_series_covid19_recovered_global_long = pd.melt(time_series_covid19_recovered_global, id_vars=idVars, var_name='Date', value_name='Case')
time_series_covid19_recovered_global_long.insert(0, 'CaseType', 'recovered')

#combined
time_series_covid19_global_long = pd.concat([time_series_covid19_confirmed_global_long, time_series_covid19_deaths_global_long, time_series_covid19_recovered_global_long])
time_series_covid19_global_long = time_series_covid19_global_long.reset_index(drop=True)

In [ ]:
##Data Transposition for US data

#confirmed_US
idVars = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key']
time_series_covid19_confirmed_US_long = pd.melt(time_series_covid19_confirmed_US, id_vars=idVars, var_name='Date', value_name='Case')
time_series_covid19_confirmed_US_long.insert(0, 'CaseType', 'confirmed')

#deaths_US
time_series_covid19_deaths_US = time_series_covid19_deaths_US.drop(labels='Population', axis=1)
idVars = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key']
time_series_covid19_deaths_US_long = pd.melt(time_series_covid19_deaths_US, id_vars=idVars, var_name='Date', value_name='Case')
time_series_covid19_deaths_US_long.insert(0, 'CaseType', 'deaths')

#combined
time_series_covid19_US_long = pd.concat([time_series_covid19_confirmed_US_long, time_series_covid19_deaths_US_long])

In [ ]:
#Transform Date from str to date
time_series_covid19_global_long = time_series_covid19_global_long.assign(Date=lambda x: pd.to_datetime(x.Date))
time_series_covid19_US_long = time_series_covid19_US_long.assign(Date=lambda x: pd.to_datetime(x.Date))

In [ ]:
#replace NA
filled_province_state = [country if pd.isna(province) else province for country, province in zip(time_series_covid19_global_long['Country/Region'].values, time_series_covid19_global_long['Province/State'].values)]
time_series_covid19_global_long = time_series_covid19_global_long.drop('Province/State', axis=1)
time_series_covid19_global_long.insert(1, 'Province/State', filled_province_state)

In [ ]:
# global time series data
time_series_covid19_global_long = time_series_covid19_global_long.sort_values(['Province/State', 'Country/Region', 'CaseType', 'Date']).reset_index(drop=True)
groupby_obj = time_series_covid19_global_long.groupby(['CaseType', 'Country/Region'])
case_shifted = groupby_obj.shift(1)['Case'].values
case_shifted = np.where(np.isnan(case_shifted), 0, case_shifted)
daily_increase = time_series_covid19_global_long['Case'].values - case_shifted
time_series_covid19_global_long['Daily_Increase'] = daily_increase

In [ ]:
# US time series data
time_series_covid19_US_long = time_series_covid19_US_long.sort_values(['Admin2', 'Province_State', 'CaseType', 'Date']).reset_index(drop=True)
groupby_obj = time_series_covid19_US_long.groupby(['CaseType', 'Combined_Key'])
case_shifted = groupby_obj.shift(1)['Case'].values
case_shifted = np.where(np.isnan(case_shifted), 0, case_shifted)
daily_increase = time_series_covid19_US_long['Case'].values - case_shifted
time_series_covid19_US_long['Daily_Increase'] = daily_increase